# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [17]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [18]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

// Removing loops reduces complexity and hallucinations
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = false });

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [19]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "samples", "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [20]:
#pragma warning disable SKEXP0060

var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 1: Define date idea options --}}
{{set "dateIdeas" (concat "We can" " " "go to a museum," " " "go for a walk in the park," " " "read poetry in a coffee shop," " " "have a picnic in the park," " " "write a poem for each other," " " "attend an open poetry slam night," " " "take a dance class together," " " "have a romantic candlelit dinner at home," " " "visit a wine tasting.")}}

{{!-- Step 2: Select three date ideas --}}
{{set "selectedIdeas" (WriterPlugin-Brainstorm input="valentine's day date ideas")}}

{{!-- Step 3: Generate a poem using the selected date ideas --}}
{{set "valentinePoem" (WriterPlugin-ShortPoem input=(json (concat "Some nice ideas " " " (json selectedIdeas))))}}

{{!-- Step 4: Output the poem created in Step 3 --}}
{{json valentinePoem}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [21]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [22]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 1: Initialize the variables --}}
{{set "event" "Valentine's day"}}
{{set "ideas" (concat "Perhaps tonight we shall enjoy a play,\n" "Amongst the many works of Shakespeare,\n" "Or take a stroll along the bay,\n" "And let the moon's light our path make clear,\n" "Or we could dine beneath the starry sky,\n" "And nibble on the finest feast,\n" "And of our fondness make the world a little shy,\n" "As happy memories do we release.\n")}}
{{!-- Step 2: Print the final result using the json helper. --}}
{{json (concat "For " event " I have considered a few options. Let me share these with you in the bard's own words:\n\n" ideas)}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [23]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

For a romantic dinner, don't be shy
Pick a fancy restaurant, give it a try
Order some wine, and a
fancy dish
And don't forget to make a wish

A picnic in the park, oh what fun
Homemade snacks, for
everyone
Lay out a blanket



Now lets execute and print the new plan:

In [24]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);

New Plan results:

For Valentine's day I have considered a few options. Let me share these with you in the bard's own words:\n\nPerhaps tonight we shall enjoy a play,\nAmongst the many works of Shakespeare,\nOr take a stroll along the bay,\nAnd let the moon's light our path make clear,\nOr we could dine beneath the starry sky,\nAnd nibble on the finest feast,\nAnd of our fondness make the world a little shy,\nAs happy memories do we release.\n


### Observations

* Hallucinated helpers break the plan.
* The plans can generate garbage, which can break the plan.
* Removing loops seems to reduce hallucinations since the complexity goes down a bit. Doesn't always listen though.